# Merging datasets

In [133]:
import pandas as pd

In [134]:
path_mercado = 'C:/workspace/Desafio Grão Direto IA/Data/mercado-desafio-tratado.xlsx'
path_transacoes = 'C:/workspace/Desafio Grão Direto IA/Data/transações-desafio-tratado.xlsx'

mercado = pd.read_excel(path_mercado)
transacoes =  pd.read_excel(path_transacoes)

In [135]:
seller_transactioned_sum = transacoes.groupby('Seller ID')['Transactioned'].sum()
seller_ids = seller_transactioned_sum[seller_transactioned_sum > 2].index
transacoes = transacoes[transacoes['Seller ID'].isin(seller_ids)]

In [136]:
model_df = pd.merge(transacoes,mercado,on='Date',how='left')

In [137]:
model_df

,Date,Seller ID,Last Transaction,Last Week,Last Month,Day of Week,Transactioned,Price,CBOT,Close,Change,Percentage Change
0,2024-01-30,100000060,0,0,0,1,0,97.111044,1215.886963,4.950200,False,0.000000
1,2024-01-31,100000060,0,0,0,2,0,98.585604,1222.299024,4.950300,True,0.002023
2,2024-02-01,100000060,0,0,0,3,0,96.726709,1209.233105,4.951900,True,0.032307
3,2024-02-02,100000060,0,0,0,4,0,97.929889,1199.996115,4.915000,False,-0.750764
4,2024-02-03,100000060,0,0,0,5,0,97.929889,1199.996115,4.915000,False,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
186298,2024-11-01,100000832,240,0,0,4,0,117.601042,1012.899216,5.789400,True,0.468098
186299,2024-11-02,100000832,241,0,0,5,0,117.601042,1012.899216,5.789400,False,0.000000
186300,2024-11-03,100000832,242,0,0,6,0,117.601042,1012.899216,5.789400,False,0.000000
186301,2024-11-04,100000832,243,0,0,0,0,115.506245,1020.956401,5.677526,False,-1.970473


# Separando o dataset com temporal_train_test

In [138]:
# from sktime.forecasting.model_selection import temporal_train_test_split

In [139]:
# seller_id = 100000060
# time_serie = model_df[model_df['Seller ID'] == seller_id]
# time_serie.drop(columns=['Seller ID','Day of Week','Last Week','Last Month'],inplace=True)
# time_serie['Change'] = time_serie['Change'].astype('Int64')
# time_serie

# Lag de um dia na nossa time serie

In [140]:
# time_serie.set_index('Date',inplace=True)

In [141]:
# target = time_serie.pop('Transactioned')
# y = pd.DataFrame(data=target,columns=['Transactioned'])
# y

In [142]:
# time_serie = time_serie.shift(1)
# time_serie.drop(pd.to_datetime('2024-01-30'), inplace=True)
# time_serie

In [143]:
# date_to_pred = pd.to_datetime('2024-11-04')
# time_serie = time_serie[time_serie.index < date_to_pred]
# y = y[y.index<date_to_pred]
# y.drop(pd.to_datetime('2024-01-30'), inplace=True)
# y

In [144]:
# time_serie

In [145]:
# y_train, y_test, X_train, X_test = temporal_train_test_split(y,time_serie,test_size=20)

# Creating forecasting

In [146]:
# from sktime.forecasting.base import ForecastingHorizon
# import numpy as np

# fh = ForecastingHorizon(np.arange(1,len(y_test)+2),is_relative=True)

In [147]:
# from xgboost import XGBRegressor

# # Calculando scale_pos_weight
# n_pos = len(y_train[y_train['Transactioned'] == 1])
# n_neg = len(y_train[y_train['Transactioned'] == 0])
# scale_pos_weight = n_neg / n_pos

# xgb_model = XGBRegressor(
#     n_estimators=100,
#     learning_rate=0.1,
#     max_depth=5,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     random_state=42,
#     scale_pos_weight=scale_pos_weight
# )

In [148]:
# from sktime.forecasting.compose import make_reduction

# forecaster = make_reduction(xgb_model, window_length=12, strategy="recursive")

In [149]:
# X_train.info()

In [150]:
# forecaster.fit(y=y_train,X=X_train)

In [151]:
# y_pred = forecaster.predict(fh=fh,X=X_test)
# y_pred

In [152]:
# from sktime.utils.plotting import plot_series
# plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])

In [153]:
# # Análise de erros
# errors = abs(y_test['Transactioned'].values - y_pred.values)
# print("Erro Médio Absoluto (MAE):", errors.mean())

# Iterando em sellers

In [154]:
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
from xgboost import XGBRegressor
from sktime.forecasting.compose import make_reduction
from sktime.utils.plotting import plot_series
import numpy as np
import matplotlib.pyplot as plt

# Lista de Seller IDs
seller_ids = model_df['Seller ID'].unique()

# Resultados para armazenar os erros por vendedor
results = []

for seller_id in seller_ids:
    print(f"Processando Seller ID: {seller_id}")
    
    # Filtrar os dados para o Seller ID atual
    time_serie = model_df[model_df['Seller ID'] == seller_id]
    time_serie.drop(columns=['Seller ID', 'Day of Week', 'Last Week', 'Last Month'], inplace=True)
    time_serie['Change'] = time_serie['Change'].astype('Int64')
    
    # Preparação da série temporal
    time_serie.set_index('Date', inplace=True)
    target = time_serie.pop('Transactioned')
    y = pd.DataFrame(data=target, columns=['Transactioned'])
    time_serie = time_serie.shift(1)
    
    # Remover valores após a data de previsão
    date_to_pred = pd.to_datetime('2024-11-04')
    time_serie = time_serie[time_serie.index < date_to_pred]
    y = y[y.index < date_to_pred]
    
    # Dividir os dados em treino e teste
    y_train, y_test, X_train, X_test = temporal_train_test_split(y, time_serie, test_size=20)
    
    # Configuração do Forecasting Horizon
    fh = ForecastingHorizon(np.arange(1, len(y_test) + 2), is_relative=True)
    
    # Calculando scale_pos_weight
    n_pos = len(y_train[y_train['Transactioned'] == 1])
    n_neg = len(y_train[y_train['Transactioned'] == 0])
    scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1
    
    # Configurar o XGBRegressor
    xgb_model = XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        scale_pos_weight=scale_pos_weight
    )
    
    # Criar e treinar o modelo
    forecaster = make_reduction(xgb_model, window_length=12, strategy="recursive")
    forecaster.fit(y=y_train, X=X_train)
    
    # Previsão
    y_pred = forecaster.predict(fh=fh, X=X_test)
    y_pred_04 = y_pred.loc['2024-11-04']
    
    # Análise de erros
    errors = abs(y_test['Transactioned'].values - y_pred_04.values)
    mae = errors.mean()
    print(f"Erro Médio Absoluto (MAE) para Seller ID {seller_id}: {mae}")
    
    # # Plot das séries
    # plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"])
    # plt.title(f"Previsão para Seller ID: {seller_id}")
    # plt.show()
    
    # Armazenar os resultados
    results.append({
        "Seller ID": seller_id,
        "Score": y_pred_04,
        "MAE": mae
    })

# Criar DataFrame com os resultados
results_df = pd.DataFrame(results)
results_df

Processando Seller ID: 100000060
Erro Médio Absoluto (MAE) para Seller ID 100000060: 0.04826273396611214
Processando Seller ID: 100002332


C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_8516\2327759639.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_serie.drop(columns=['Seller ID', 'Day of Week', 'Last Week', 'Last Month'], inplace=True)
C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_8516\2327759639.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_serie['Change'] = time_serie['Change'].astype('Int64')
C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_8516\2327759639.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

Erro Médio Absoluto (MAE) para Seller ID 100002332: 0.1
Processando Seller ID: 100002035
Erro Médio Absoluto (MAE) para Seller ID 100002035: 0.11761491447687149
Processando Seller ID: 100001525


C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_8516\2327759639.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_serie.drop(columns=['Seller ID', 'Day of Week', 'Last Week', 'Last Month'], inplace=True)
C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_8516\2327759639.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_serie['Change'] = time_serie['Change'].astype('Int64')
C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_8516\2327759639.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

Erro Médio Absoluto (MAE) para Seller ID 100001525: 0.05431249151006341
Processando Seller ID: 100000192
Erro Médio Absoluto (MAE) para Seller ID 100000192: 0.03232159838080406
Processando Seller ID: 100000310


...

C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_8516\2327759639.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_serie.drop(columns=['Seller ID', 'Day of Week', 'Last Week', 'Last Month'], inplace=True)
C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_8516\2327759639.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_serie['Change'] = time_serie['Change'].astype('Int64')
C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_8516\2327759639.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

Erro Médio Absoluto (MAE) para Seller ID 100000538: 9.051207598531619e-06
Processando Seller ID: 100000559
Erro Médio Absoluto (MAE) para Seller ID 100000559: 1.5347717635449953e-05
Processando Seller ID: 100000832


C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_8516\2327759639.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_serie.drop(columns=['Seller ID', 'Day of Week', 'Last Week', 'Last Month'], inplace=True)
C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_8516\2327759639.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_serie['Change'] = time_serie['Change'].astype('Int64')
C:\Users\Miguel Mendes\AppData\Local\Temp\ipykernel_8516\2327759639.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

Erro Médio Absoluto (MAE) para Seller ID 100000832: 1.5505500414292328e-05


,Seller ID,Score,MAE
0,100000060,Transactioned 0.048263 Name: 2024-11-04 00:...,0.048263
1,100002332,Transactioned 0.0 Name: 2024-11-04 00:00:00...,0.100000
2,100002035,Transactioned 0.022019 Name: 2024-11-04 00:...,0.117615
3,100001525,Transactioned -0.004312 Name: 2024-11-04 00:...,0.054312
4,100000192,Transactioned -0.032322 Name: 2024-11-04 00:...,0.032322
...,...,...,...
658,100000993,Transactioned 0.000008 Name: 2024-11-04 00:...,0.000008
659,100001006,Transactioned 0.000014 Name: 2024-11-04 00:...,0.000014
660,100000538,Transactioned 0.000009 Name: 2024-11-04 00:...,0.000009
661,100000559,Transactioned 0.000015 Name: 2024-11-04 00:...,0.000015


In [155]:
users_that_made_transaction_on_target_date = [
    100000060,
    100000141,
    100000192,
    100000310,
    100000348,
    100001098,
    100001362,
    100001525,
    100002035,
    100002271,
    100002332,
    100002339,
    100002402,
    100002426,
]

In [159]:
first_200_predictions = results_df.sort_values(
    ascending=False, by="MAE"
).head(200)


first_200_predictions[
    first_200_predictions["Seller ID"].isin(users_that_made_transaction_on_target_date)
]

,Seller ID,Score,MAE
6,100000348,Transactioned -0.041753 Name: 2024-11-04 00:...,0.391753
2,100002035,Transactioned 0.022019 Name: 2024-11-04 00:...,0.117615
7,100001098,Transactioned 0.013613 Name: 2024-11-04 00:...,0.110891
1,100002332,Transactioned 0.0 Name: 2024-11-04 00:00:00...,0.100000
5,100000310,Transactioned 0.080554 Name: 2024-11-04 00:...,0.080554
